# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../WeatherPy/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,zayed city,24.3768,54.5890,34.08,46,0,3.60,AE,1727727368
1,1,iqaluit,63.7506,-68.5145,7.85,65,75,3.60,CA,1727727369
2,2,lapeer,43.0514,-83.3188,23.84,60,28,4.52,US,1727727371
3,3,badger,64.8000,-147.5333,7.77,59,20,0.89,US,1727727372
4,4,blackmans bay,-43.0167,147.3167,12.63,59,31,1.34,AU,1727727374


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles='OSM',
    frame_width=1200,
    frame_height=800,
    size = 'Humidity',
    color = 'City'

    
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df.loc[((city_data_df["Max Temp"] < 26) & (city_data_df["Max Temp"] > 21)) &(city_data_df["Wind Speed"] < 10) & (city_data_df["Cloudiness"] <5)]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,11,maintirano,-18.0667,44.0167,24.93,78,0,1.55,MG,1727727385
16,16,bethel,41.3712,-73.4140,22.43,65,0,5.14,US,1727727392
43,43,calama,-22.4667,-68.9333,23.27,9,0,7.72,CL,1727727314
92,92,roma,43.2128,-75.4557,23.53,62,0,2.57,US,1727727241
121,121,tolanaro,-25.0319,46.9987,23.04,83,0,8.23,MG,1727727548
143,143,pacific grove,36.6177,-121.9166,23.57,80,0,3.58,US,1727727581
152,152,lompoc,34.6391,-120.4579,25.29,61,3,4.12,US,1727727594
203,203,paros,37.0833,25.1500,21.84,63,0,2.57,GR,1727727668
216,216,tortoli,39.9256,9.6572,21.10,86,0,2.57,IT,1727727689
245,245,saint-hyacinthe,45.6168,-72.9491,23.34,63,0,3.11,CA,1727727732


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotels_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotels_df["Hotel Name"] = ""

# Display sample data
hotels_df.head(20)

,City,Country,Lat,Lng,Humidity,Hotel Name
0,zayed city,AE,24.3768,54.5890,46,
1,iqaluit,CA,63.7506,-68.5145,65,
2,lapeer,US,43.0514,-83.3188,60,
3,badger,US,64.8000,-147.5333,59,
4,blackmans bay,AU,-43.0167,147.3167,59,
5,olonkinbyen,SJ,70.9221,-8.7187,64,
6,cabedelo,BR,-6.9811,-34.8339,72,
7,port elizabeth,ZA,-33.9180,25.5701,87,
8,saint-philippe,RE,-21.3585,55.7679,80,
9,waitangi,NZ,-43.9535,-176.5597,78,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "radius": radius,
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotels_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params) 

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotels_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        #If no hotel is found, set the hotel name as "No hotel found".
        hotels_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotels_df.loc[index, 'City']} - nearest hotel: {hotels_df.loc[index, 'Hotel Name']}")

# Display sample data
hotels_df

Starting hotel search
zayed city - nearest hotel: L'Arabia hotel
iqaluit - nearest hotel: Frobisher Inn
lapeer - nearest hotel: Holiday Inn Express Lapeer
badger - nearest hotel: Clarion Hotel & Suites
blackmans bay - nearest hotel: Villa Howden
olonkinbyen - nearest hotel: No hotel found
cabedelo - nearest hotel: No hotel found
port elizabeth - nearest hotel: Waterford Hotel
saint-philippe - nearest hotel: Le Baril
waitangi - nearest hotel: Hotel Chathams
cumana - nearest hotel: Hotel Regina
maintirano - nearest hotel: Hotel Safari
ribeira grande - nearest hotel: Hospedaria JSF
vallenar - nearest hotel: Hotel del Marqués
talnakh - nearest hotel: Талнах
labasa - nearest hotel: Grand Eastern Hotel
bethel - nearest hotel: Hampton Inn Danbury
toliara - nearest hotel: Ambary
puerto natales - nearest hotel: Dorotea Patagonia Hostel
mayna - nearest hotel: No hotel found
port-aux-francais - nearest hotel: Keravel
longyearbyen - nearest hotel: The Vault
reggane - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
0,zayed city,AE,24.3768,54.5890,46,L'Arabia hotel
1,iqaluit,CA,63.7506,-68.5145,65,Frobisher Inn
2,lapeer,US,43.0514,-83.3188,60,Holiday Inn Express Lapeer
3,badger,US,64.8000,-147.5333,59,Clarion Hotel & Suites
4,blackmans bay,AU,-43.0167,147.3167,59,Villa Howden
...,...,...,...,...,...,...
557,tazacorte,ES,28.6290,-17.9293,91,App Leyma
558,udachny,RU,66.4167,112.4000,99,Вилюй
559,akureyri,IS,65.6835,-18.0878,80,Hótel Norðurland
560,polyarnyy,RU,69.1989,33.4478,73,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot
map_plot2 = hotels_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles='OSM',
    frame_width=1200,
    frame_height=800,
    size = 'Humidity',
    color = 'Hotel Name',
    hover_cols = ["City", "Country", "Hotel Name"]
)
# Display the map
map_plot2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Humidity,City,Country)